In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from google.colab.patches import cv2_imshow
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from keras.applications import EfficientNetB0


In [3]:
# train -> train0+train1+train2
train_path="/content/gdrive/MyDrive/TRAIN_UP"
# test -> test0+test1+test2
test_path="/content/gdrive/MyDrive/TEST_UP"
class_names=os.listdir(train_path)
class_names_test=os.listdir(test_path)

In [4]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)

In [5]:
train_generator = train_datagen.flow_from_directory("/content/gdrive/My Drive/TRAIN_UP")
test_generator = test_datagen.flow_from_directory("/content/gdrive/My Drive/TEST_UP")


Found 1392 images belonging to 3 classes.
Found 602 images belonging to 3 classes.


In [6]:

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the pre-trained EfficientNet model without the top layer
base_model = EfficientNetB0(weights='imagenet', input_shape=(224, 224, 3),include_top=False)

# Add a new top layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Define the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False
model.summary()
# Compile the model
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
train_generator = train_datagen.flow_from_directory(
    '/content/gdrive/My Drive/TRAIN_UP',
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    '/content/gdrive/My Drive/TEST_UP',
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical')
H = model.fit(train_generator,validation_data =test_generator, epochs=40,verbose=1)

16705208/16705208 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)     

KeyboardInterrupt: ignored

In [ ]:
model.evaluate_generator(test_generator)

<ipython-input-7-faa0e9406f88>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator)


[0.4279547333717346, 0.8189368844032288]

In [ ]:
from keras.models import model_from_json

In [ ]:
model.save_weights('/content/gdrive/My Drive/EN_Weights_UP.h5')

In [ ]:
mkdir -p saved_model

In [ ]:
model.save('saved_model/my_model')

In [ ]:
# Saving model weights into drive
#model.save("/content/gdrive/My Drive/EN_Weights_UP.h5")
import json
model_json = model.to_json()
with open("/content/gdrive/My Drive/EN.json","w") as json_file:
  json_file.write(str(model_json))

In [ ]:
import pandas as pd
hist_df = pd.DataFrame(H.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
mc = ModelCheckpoint('/content/gdrive/My Drive/EN.h5', monitor='val_accuracy')

In [ ]:
def predict_(image_path):
  #Load the Model from Json File
  json_file = open('/content/gdrive/My Drive/EN.json', 'r')
  model_json_c = json_file.read()
  json_file.close()
  model_c = model_from_json(model_json_c)
  #Load the weights
  model_c.load_weights("/content/gdrive/My Drive/EN.h5")
  #Compile the model
  opt = SGD(lr=1e-4, momentum=0.9)
  model_c.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
  #load the image you want to classify
  image = cv2.imread(image_path)
  image = cv2.resize(image, (224,224))
  cv2_imshow(image)
  #predict the image
  preds = model_c.predict(np.expand_dims(image, axis=0))[0]
  prediction = max(preds)
  print(preds)

  if prediction==preds[0]:
      print("Predicted Label: 0")
  elif prediction==preds[1]:
      print("Predicted Label: 1")
  else:
      print("Predicted Label: 2")


In [ ]:
predict_("/content/gdrive/My Drive/TEST_UP/Test_img_0/17023_right.jpeg")

In [ ]:
predict_("/content/gdrive/MyDrive/train.zip/train/28780_right.jpeg")

In [ ]:
hist_df

In [ ]:
import matplotlib.pyplot as plt
loss_train = hist_df['loss']
loss_val = hist_df['val_loss']
epochs = range(40)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = hist_df['accuracy']
loss_val = hist_df['val_accuracy']
epochs = range(40)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
true_value = []
predicted_value = []
plt.figure(figsize=(10,10))
import os
from os import listdir
# get the path or directory
testImages = []
folder_dir = "/content/gdrive/MyDrive/test/DB0_test"
for i in os.listdir(folder_dir):
    # check if the image ends with png or jpg or jpeg
    if (i.endswith(".png") or i.endswith(".jpg")\
        or i.endswith(".jpeg")):
        testImages.append(folder_dir+'/'+i)
        true_value.append(0)
folder_dir = "/content/gdrive/MyDrive/test/DB1_test"
for i in os.listdir(folder_dir):
    # check if the image ends with png or jpg or jpeg
    if (i.endswith(".png") or i.endswith(".jpg")\
        or i.endswith(".jpeg")):
        testImages.append(folder_dir+'/'+i)
        true_value.append(1)``
folder_dir = "/content/gdrive/MyDrive/test/DB2_test"
for i in os.listdir(folder_dir):
    # check if the image ends with png or jpg or jpeg
    if (i.endswith(".png") or i.endswith(".jpg")\
        or i.endswith(".jpeg")):
        testImages.append(folder_dir+'/'+i)
        true_value.append(2)


def predict_(image_path):
  #Load the Model from Json File
  json_file = open('/content/gdrive/My Drive/model.json', 'r')
  model_json_c = json_file.read()
  json_file.close()
  model_c = model_from_json(model_json_c)
  #Load the weights
  model_c.load_weights("/content/gdrive/My Drive/best_model.h5")
  #Compile the model
  opt = SGD(lr=1e-4, momentum=0.9)
  model_c.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
  #load the image you want to classify
  image = cv2.imread(image_path)
  image = cv2.resize(image, (224,224))
  #cv2_imshow(image)
  #predict the image
  preds = model_c.predict(np.expand_dims(image, axis=0))[0]
  prediction = max(preds)
  #print(preds)

  if prediction==preds[0]:
      #print("Predicted Label: 0")
      predicted_value.append(0)
  elif prediction==preds[1]:
      #print("Predicted Label: 1")
      predicted_value.append(1)
  else:
      #print("Predicted Label: 2")
      predicted_value.append(2)


for image in testImages:
  print(image)
  predict_(image)





In [ ]:
li = []
for i in range(len(predicted_value)):
  li.append(i+1)
for i in range(len(predicted_value)):
  li[i] = li[i]+(predicted_value[i]-true_value[i])*10
Li = []
for i in range(i+1):
  Li.append(i+1)

plt.figure(figsize=(20,20))
plt.scatter(Li, li, c ="red",
            linewidths =1,
            marker ="*",
            edgecolor ="green",
            s = 200)
plt.scatter(Li, Li, c ="blue",
            linewidths =0.11,
            marker =".",
            edgecolor ="blue",
            s = 20)
plt.show()
